In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

/Users/rajsingh/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


We have 9 input variables, and will create a 3-layer neural network with 50 nodes, 100 nodes, and 50 nodes respectively. We are trying to predict the 'total earnings' column.

In [2]:
# define model parameters
learning_rate = 0.001
training_epochs = 100
display_step = 5

# define number of inputs and outputs
number_of_inputs = 9
number_of_outputs = 1

# define how many neurons we want in each layer of the network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50



# Define layers

# input layer 
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32,shape=(None,number_of_inputs))
    
# layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable('weights1',shape=[number_of_inputs,layer_1_nodes],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases1',shape=[layer_1_nodes],initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X,weights)+biases)
# layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable('weights2',shape=[layer_1_nodes,layer_2_nodes],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases2',shape=[layer_2_nodes],initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output,weights)+biases)
# layer 3    
with tf.variable_scope('layer_3'):
    weights = tf.get_variable('weights3',shape=[layer_2_nodes,layer_3_nodes],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases3',shape=[layer_3_nodes],initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output,weights)+biases)
# output layer
with tf.variable_scope('output'):
    weights = tf.get_variable('weights4',shape=[layer_3_nodes,number_of_outputs],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases4',shape=[number_of_outputs],initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output,weights)+biases
    
# define cost function
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32,shape=(None,1))
    cost = tf.reduce_mean(tf.squared_difference(prediction,Y))
    
# define optimizer
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# initialize session
with tf.Session() as session:
    # initializes all variables 
    session.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        
        # do one round of training
        session.run(optimizer,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
        
        # log training progress every 5 steps
        if epoch % 5 ==0:
            training_cost = session.run(cost,feed_dict={X:X_scaled_training, Y:Y_scaled_training})
            testing_cost = session.run(cost,feed_dict={X:X_scaled_testing,Y:Y_scaled_testing})
            print(epoch,training_cost,testing_cost)
            
    print("training complete")
    
    final_training_cost = session.run(cost,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
    final_testing_cost = session.run(cost,feed_dict={X:X_scaled_testing,Y:Y_scaled_testing})
    
    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

(0, 0.21781875, 0.23126331)
(5, 0.028943483, 0.03147338)
(10, 0.04032844, 0.041138917)
(15, 0.022949705, 0.023837551)
(20, 0.013487322, 0.014731992)
(25, 0.013733491, 0.014897444)
(30, 0.0103747165, 0.0111595355)
(35, 0.0075028106, 0.008038129)
(40, 0.0070414725, 0.0075469483)
(45, 0.005451291, 0.005968302)
(50, 0.004533495, 0.005026658)
(55, 0.0038065643, 0.0042511933)
(60, 0.0029724974, 0.0033567292)
(65, 0.0024093052, 0.002758203)
(70, 0.0019117111, 0.0022243294)
(75, 0.0015029416, 0.0017614645)
(80, 0.0012466385, 0.0014402182)
(85, 0.0010537746, 0.0011755807)
(90, 0.00086624315, 0.0009996642)
(95, 0.000723104, 0.00085591554)
training complete
Final Training cost: 0.000636415032204
Final Testing cost: 0.00074935820885
